## Imports

In [1]:
import asyncio
import time
from threading import Thread
from rtlsdr import RtlSdr


## Setup logging

In [2]:
import logging
import os
#Cleanup
try:
    os.remove('mylog.log')
except:
    pass
#loggingSetup
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

## Global variables

In [3]:
#global sdr
exit_flag = False
upd_flag = False

n = 128*1024

sample_rate = 2.4e6
center_freq = 99.5e6
sdr_gain = 496

buffer_received = False

# PL setup

## Setup Overlay

In [4]:
from pynq import Overlay
import pynq.lib.dma
# Load the overlay

In [5]:
overlay = Overlay('/home/xilinx/pynq/overlays/Teste/design_1.bit')
# Load the DMA
#dma = overlay.axi_dma
pAudio = overlay.audio_codec

## Setup DMA buffers

In [6]:
from pynq import Xlnk
import numpy as np
import math
xlnk = Xlnk()
#send_buffer = xlnk.cma_array(shape=(n,), dtype=np.int8) #Allocate input buffer 
#rec_buffer = xlnk.cma_array(shape=(n,), dtype=np.int32) # Allocate output buffer

## Setup Audio

In [7]:
# Buffer setup
pl_smpl = int(64e3)
pAudio.sample_len = math.ceil(pl_smpl)
num_samples_32b = pAudio.sample_len * 2
# Create data buffer
temp_buffer = np.zeros(num_samples_32b, dtype=np.int32)
pAudio.buffer = np.zeros(num_samples_32b, dtype=np.int32)

## Define functions

In [8]:
#Define streaming function
async def streaming(sdr):
    logging.debug("Inside streaming")
    async for raw_samples in sdr.stream(num_samples_or_bytes=n, format="bytes"):
        if not exit_flag:
            logging.debug(raw_samples)
            np.copyto(send_buffer, raw_samples)
            logging.debug("Transferring the buffer to PL")
            dma.sendchannel.transfer(send_buffer)
            #dma.sendchannel.wait() #Can we do this with async.wait? YES
            #settup interrupt
            logging.debug("Awaiting for Tx transfer to finish")
            await dma.sendchannel.wait_async()
            logging.debug("Tx transfer finished")
            # clear the buffer?
        else:
            logging.debug("Stopping SDR")
            break
    #Cleanup
    logging.debug("Running sdr.stop()")
    await sdr.stop()
    sdr.close()
    logging.debug("Exiting streaming")

In [9]:
# Receive and play demodulated samples
async def receiving():
    global buffer_received
    while not exit_flag: 
        logging.debug("Receiving data from PL")
        dma.recvchannel.transfer(rec_buffer)
        await dma.recvchannel.wait_async()
        logging.debug("Data from PL received")
        #np.copyto(pAudio.buffer, [rec_buffer, rec_buffer])
        #np.copyto(pAudio.buffer, np.concatenate((rec_buffer, rec_buffer), axis=None))
        np.copyto(temp_buffer, np.repeat(rec_buffer*5, 2))
        #await asyncio.sleep(1)
        buffer_received = True

In [10]:
#Define RTL-SDR update function
async def sdr_upd(sdr):
    global upd_flag
    while not exit_flag:
        if upd_flag:
            logging.debug("sample_rate => " + str(sample_rate))
            logging.debug("center_freq => " + str(center_freq))
            logging.debug("sdr_gain => " + str(sdr_gain))
            sdr.sample_rate = sample_rate
            sdr.center_freq = center_freq
            sdr.sdr_gain    = sdr_gain
            await asyncio.sleep(1)
            logging.debug("sample_rate = " + str(sdr.get_sample_rate()))
            logging.debug("center_freq = " + str(sdr.get_center_freq()))
            logging.debug("sdr_gain = " + str(sdr.get_gain()))
            upd_flag = False
        await asyncio.sleep(1)

In [11]:
#Thread function
def start_threaded_loop():
    logging.debug("Starting the stream")
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    global dma
    global send_buffer
    global rec_buffer
    dma = overlay.axi_dma
    send_buffer = xlnk.cma_array(shape=(n,), dtype=np.int8) #Allocate input buffer 
    rec_buffer = xlnk.cma_array(shape=(pl_smpl,), dtype=np.int32) # Allocate output buffer
    
    try:
        logging.debug("Starting new loop")
        sdr = RtlSdr()
        loop.run_until_complete(asyncio.gather(streaming(sdr), receiving(), sdr_upd(sdr)))
    finally:
        logging.debug("Closing the loop")
        loop.close()
        logging.debug("Loop closed \n returning")

### Define function for playing audio

In [12]:
def playing_thread():
    global buffer_received
    while not exit_flag:
        if buffer_received:
            np.copyto(pAudio.buffer, temp_buffer)
            buffer_received = False
            logging.debug("Playing")
            pAudio.play()
            logging.debug("Played")

## Create and start a seperate thread for streaming

In [13]:
nthread = Thread(target=start_threaded_loop)
pthread = Thread(target=playing_thread)
logging.debug("Starting new thread")
nthread.start()
pthread.start()
time.sleep(10)

## RTL-SDR setup

In [14]:
upd_flag = True
center_freq = 99.5e6
sdr_gain = 496
logging.debug("Updating RTL-SDR in main")

In [ ]:
time.sleep(5)

## Kill it with fire

In [ ]:
logging.debug("EXIT_FLAG")
exit_flag = True
time.sleep(5)
nthread.join()
logging.debug("nthread is alive: " + str(nthread.isAlive()))